In [1]:
import os, re, glob, numpy as np, torch, torch.nn as nn, torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

In [2]:
def compute_global_image_stats(chunk_paths, sample_fraction=0.1):
    sum_pixels = np.zeros(3, dtype=np.float64)
    sum_sq_pixels = np.zeros(3, dtype=np.float64)
    total_pixels = 0
    for chunk in chunk_paths:
        data = np.load(chunk)
        images = data['X_jets']
        N = images.shape[0]
        if sample_fraction < 1.0:
            sample_size = max(1, int(N*sample_fraction))
            idx = np.random.choice(N, sample_size, replace=False)
            images = images[idx]
        pixels = images.reshape(-1,3).astype(np.float64)
        sum_pixels += pixels.sum(axis=0)
        sum_sq_pixels += (pixels**2).sum(axis=0)
        total_pixels += pixels.shape[0]
        del data, images, pixels
    mean = sum_pixels/total_pixels; std = np.sqrt(sum_sq_pixels/total_pixels - mean**2)
    return mean.tolist(), std.tolist()

In [3]:
class JetImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images; self.labels = labels; self.transform = transform
    def __len__(self): return len(self.images)
    def __getitem__(self, idx):
        img = self.images[idx]; label = self.labels[idx]
        if self.transform: img = self.transform(img)
        return img, label


In [4]:
def compute_physics_features_tensor(x, eps=1e-6):
    ecal = x[:,0:1,:,:]; hcal = x[:,1:2,:,:]; tracks = x[:,2:3,:,:]
    ratio = torch.mean(ecal/(hcal+eps), dim=[2,3])
    mean_tracks = torch.mean(tracks, dim=[2,3])
    diff = torch.mean(ecal-hcal, dim=[2,3])
    return torch.cat([ratio, mean_tracks, diff], dim=1)

In [5]:
class ChannelWiseConvBranch(nn.Module):
    def __init__(self, in_channels=1, out_channels=8):
        super().__init__()
        self.convs = nn.ModuleList([
            nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size=1, padding=0),
                          nn.ReLU(), nn.AdaptiveAvgPool2d((1,1))),
            nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                          nn.ReLU(), nn.AdaptiveAvgPool2d((1,1))),
            nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size=5, padding=2),
                          nn.ReLU(), nn.AdaptiveAvgPool2d((1,1)))
        ])
    def forward(self, x):
        outs = [conv(x) for conv in self.convs]
        outs = [o.view(o.size(0), -1) for o in outs]
        return torch.cat(outs, dim=1)


In [6]:
class FeaturePyramidBranch(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.convs = nn.ModuleList([
            nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size=1, padding=0),
                          nn.ReLU(), nn.AdaptiveAvgPool2d((1,1))),
            nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                          nn.ReLU(), nn.AdaptiveAvgPool2d((1,1))),
            nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size=5, padding=2),
                          nn.ReLU(), nn.AdaptiveAvgPool2d((1,1)))
        ])
    def forward(self, x):
        outs = [conv(x) for conv in self.convs]
        outs = [o.view(o.size(0), -1) for o in outs]
        return torch.cat(outs, dim=1)

In [7]:
class DenseNet121PhysicsModel(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.backbone = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
        in_features = self.backbone.classifier.in_features
        self.backbone.classifier = nn.Identity()
        self.physics_fc = nn.Sequential(nn.Linear(3,16), nn.ReLU(), nn.Linear(16,16), nn.ReLU())
        self.channel_branch = ChannelWiseConvBranch(in_channels=1, out_channels=8)
        self.joint_conv = FeaturePyramidBranch(in_channels=3, out_channels=8)
        fusion_dim = in_features + 16 + (8*3*3) + (8*3)
        self.fusion_fc = nn.Sequential(nn.Linear(fusion_dim,512), nn.ReLU(), nn.Linear(512,num_classes))
    def forward(self, x):
        backbone_feat = self.backbone(x)
        phys_out = self.physics_fc(compute_physics_features_tensor(x))
        channel_feats = []
        for i in range(3):
            channel_feats.append(self.channel_branch(x[:, i:i+1, :, :]))
        channel_feats = torch.cat(channel_feats, dim=1)
        joint_feat = self.joint_conv(x)
        fused = torch.cat([backbone_feat, phys_out, channel_feats, joint_feat], dim=1)
        logits = self.fusion_fc(fused)
        return backbone_feat, logits

In [8]:
def save_checkpoint(model, optimizer, epoch, checkpoint_dir, model_name):
    os.makedirs(checkpoint_dir, exist_ok=True)
    path = os.path.join(checkpoint_dir, f"{model_name}_epoch_{epoch}.pth")
    torch.save({'epoch': epoch, 'model_state': model.state_dict(),
                'optimizer_state': optimizer.state_dict()}, path)
    print(f"Checkpoint saved: {path}")

def load_checkpoint(model, optimizer, checkpoint_dir, model_name):
    pattern = os.path.join(checkpoint_dir, f"{model_name}_epoch_*.pth")
    files = glob.glob(pattern)
    if not files:
        print(f"No checkpoint for {model_name}")
        return 0
    latest = max(files, key=lambda f: int(re.search(r'epoch_(\d+)', f).group(1)))
    ckpt = torch.load(latest)
    model.load_state_dict(ckpt['model_state'])
    optimizer.load_state_dict(ckpt['optimizer_state'])
    start_epoch = ckpt['epoch']
    print(f"Loaded {model_name} from {latest} at epoch {start_epoch}")
    return start_epoch


In [9]:
def train_model(model, device, chunk_paths, epochs=5, batch_size=32, checkpoint_dir="ckpt_densenet", resume=False):
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    start_epoch = 0
    if resume:
        start_epoch = load_checkpoint(model, optimizer, checkpoint_dir, model.__class__.__name__)
    global_mean, global_std = compute_global_image_stats(chunk_paths, sample_fraction=0.1)
    print("Global stats:", global_mean, global_std)
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((224,224)),
        transforms.Normalize(mean=global_mean, std=global_std)
    ])
    criterion = nn.CrossEntropyLoss()
    for epoch in range(start_epoch+1, epochs+1):
        print(f"\nEpoch {epoch}/{epochs}")
        for chunk in tqdm(chunk_paths, desc="Chunks"):
            print(f"\nTraining on chunk: {chunk}")
            data = np.load(chunk)
            images = data['X_jets']; labels = data['y']
            dataset = JetImageDataset(images, labels, transform=transform)
            dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            model.train()
            tot_loss = 0.0
            for imgs, labs in tqdm(dataloader, desc="Batches", leave=False):
                imgs = imgs.to(device, dtype=torch.float)
                labs = labs.to(device).long()
                optimizer.zero_grad()
                _, logits = model(imgs)
                loss = criterion(logits, labs)
                loss.backward()
                optimizer.step()
                tot_loss += loss.item()
            print(f"Chunk loss: {tot_loss/len(dataloader):.4f}")
            del data, images, labels, dataset, dataloader
        save_checkpoint(model, optimizer, epoch, checkpoint_dir, model.__class__.__name__)
    return model

In [10]:
if __name__ == '__main__':
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    data_dir = "/kaggle/input/genie-extracted-dataset"
    files = [f for f in os.listdir(data_dir) if f.endswith(".npz")]
    sorted_files = sorted(files, key=lambda f: int(re.search(r'chunk_(\d+)_', f).group(1)))
    chunk_paths = [os.path.join(data_dir, f) for f in sorted_files]
    model = DenseNet121PhysicsModel(num_classes=2)
    trained_model = train_model(model, device, chunk_paths, epochs=5, batch_size=32,
                                checkpoint_dir="ckpt_densenet", resume=True)

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 202MB/s]


No checkpoint for DenseNet121PhysicsModel
Global stats: [7.899225784395176e-05, 5.0819608753754934e-05, 3.0654087225240467e-05] [0.00523253309673214, 0.0019989281135925283, 0.0004944007743946376]

Epoch 1/5


Chunks:   0%|          | 0/14 [00:00<?, ?it/s]


Training on chunk: /kaggle/input/genie-extracted-dataset/chunk_0_10000.npz



Chunks:   7%|▋         | 1/14 [01:22<17:53, 82.60s/it]

...

Chunk loss: 0.5205

Training on chunk: /kaggle/input/genie-extracted-dataset/chunk_120000_130000.npz



Chunks:  93%|█████████▎| 13/14 [17:46<01:21, 81.95s/it]

Chunk loss: 0.5222

Training on chunk: /kaggle/input/genie-extracted-dataset/chunk_130000_139306.npz



Chunks: 100%|██████████| 14/14 [19:03<00:00, 81.65s/it]


Chunk loss: 0.5249
Checkpoint saved: ckpt_densenet/DenseNet121PhysicsModel_epoch_5.pth
